In [1]:
#Import necessary libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape Politifact website
def scrape_politifact(page_number):
    URL = 'https://www.politifact.com/factchecks/list/?page={}'.format(page_number)
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.text, "html.parser")
    statement_footer = soup.find_all('footer', attrs={'class': 'm-statement__footer'})
    statement_quote = soup.find_all('div', attrs={'class': 'm-statement__quote'})
    statement_meta = soup.find_all('div', attrs={'class': 'm-statement__meta'})
    target = soup.find_all('div', attrs={'class': 'm-statement__meter'})

    # Loop through the data and extract relevant information
    for i in range(len(statement_footer)):
        link1 = statement_footer[i].text.strip().split()
        if len(link1) >= 4:  # Ensuring array index existence
            first_name = link1[1]
            last_name = link1[2]
            full_name = first_name + ' ' + last_name
            date = ' '.join(link1[4:])
            authors.append(full_name)
            dates.append(date)

            link2 = statement_quote[i].find_all('a')
            statements.append(link2[0].text.strip())

            link3 = statement_meta[i].find_all('a')
            sources.append(link3[0].text.strip())

            fact = target[i].find('div', attrs={'class': 'c-image'}).find('img').get('alt')
            targets.append(fact)

# Create lists to store scraped data
authors = []
dates = []
statements = []
sources = []
targets = []

# Determine the number of pages to scrape
initial_page = requests.get("https://www.politifact.com/factchecks/list/")
initial_soup = BeautifulSoup(initial_page.text, "html.parser")
# Look for pagination details dynamically, or set a high number and handle empty responses
num_pages = 100  # Example: set a high number or determine dynamically

# Loop through 'n' webpages to scrape data
for i in range(1, num_pages + 1):
    scrape_politifact(i)

# Create a DataFrame
data = pd.DataFrame({
    'author': authors,
    'statement': statements,
    'source': sources,
    'date': dates,
    'target': targets
})

# Export the data to a CSV file
data.to_csv('politifact_statements.csv', index=False)
